### VQE（Variational quantum eigensolver）
パラメータ付き量子回路で変分的に基底状態を求めましょう。

### 必要なライブラリをインポート

In [ ]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit,QubitBra,measure_all,measure_partial
from sympy.physics.quantum.gate import X,Y,Z,H,CNOT,SWAP,CPHASE,CGateS
from sympy.physics.quantum.gate import IdentityGate as _I
from sympy.physics.quantum.gate import UGate as U

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import scipy.optimize
import scipy.linalg
import numpy as np
import sys

### 変分量子回路で利用するためのゲートを定義

In [ ]:
def Rxi(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*X(1)*_I(0),nqubits=2))
def Rix(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*_I(1)*X(0),nqubits=2))
def Rzi(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*Z(1)*_I(0),nqubits=2))
def Riz(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*_I(1)*Z(0),nqubits=2))

print(Rxi((0,1),pi/4).get_target_matrix())
print(Rix((0,1),pi/4).get_target_matrix())
print(Rzi((0,1),pi/4).get_target_matrix())
print(Riz((0,1),pi/4).get_target_matrix())

### ハミルトニアンの定義

$ H = \frac{1}{2} \left( S_{ii} \mathbb{1} + S_{ix} \sigma_{x}^{1} + S_{iz} \sigma_{z}^{1} + S_{xi} \sigma_{x}^{0}  + S_{zi} \sigma_{z}^{0} 
+ S_{xx} \sigma_{x}^{0} \sigma_{x}^{1} + S_{xz} \sigma_{x}^{0} \sigma_{z}^{1} + S_{zx} \sigma_{z}^{0} \sigma_{x}^{1}
+ S_{zz} \sigma_{z}^{0} \sigma_{z}^{1} \right) $

In [ ]:
Sii,Six,Sxi,Sxx,Szz,Siz,Szi,Sxz,Szx = symbols('Sii Six Sxi Sxx Szz Siz Szi Sxz Szx')
Hamiltonian = (   Sii *_I(0) *_I(1) 
                + Six *_I(0) * X(1)
                + Siz *_I(0) * Z(1)
                + Sxi * X(0) *_I(1)
                + Szi * Z(0) *_I(1)
                + Sxx * X(0) * X(1) 
                + Sxz * X(0) * Z(1)
                + Szx * Z(0) * X(1) 
                + Szz * Z(0) * Z(1) )/2
h = represent(Hamiltonian,nqubits=2)
Hamiltonian

$ HeH^+ $ 分子のハミルトニアンの数値設定（パラメータはNature Communication 2014 より)

In [ ]:
H_valued = h.subs([
    (Sii,-3.8505),
    (Six,-0.2288),
    (Sxi,-0.2288),
    (Siz,-1.0466),
    (Szi,-1.0466),
    (Sxx, 0.2613),
    (Sxz, 0.2288),
    (Szx, 0.2288),
    (Szi,-1.0466),
    (Szz,0.2356)])
H_valued

### 答えを事前に計算
このような小さいサイズのMatrixであれば、厳密対角化も計算で、固有値は求められます。

In [ ]:
# sympy は数値計算が苦手なので対角化は時間がかかります.
# ↓ sympy で提供されている対角化
# P, D = H_valued.diagonalize() 
# ↓ sympy で提供されている固有値, 固有ベクトルの求め方
E = H_valued.eigenvects()
M = np.argmin([re(E[i][0]) for i in range(len(E))])
pprint(E[M])
print(re(E[M][0]))

In [ ]:
# ↓数値計算は, 固有値を求めるのが得意な numpy でも試します. 
l, p = np.linalg.eig( np.array( H_valued.tolist(), dtype=np.complex128 ))
v = np.transpose(p)
mini = np.argmin(l)
E_answer = l[mini]
#print(p)
#print(l)
print(np.array([v[mini]]))
print(E_answer)

In [ ]:
def dice(n): return [np.random.rand() for i in range(n)]
def vqe_trial(phi):
    global count
    global f
    count += 1
    trial_func = Rxi((0,1), phi[0]).get_target_matrix() \
        * Riz((0,1), phi[1]).get_target_matrix() \
        * represent(CNOT(0,1),nqubits=2) \
        * Riz((0,1), phi[2]).get_target_matrix() \
        * Rix((0,1), phi[3]).get_target_matrix() \
        * Rzi((0,1), phi[4]).get_target_matrix() \
        * Rxi((0,1), phi[5]).get_target_matrix()
    trial_func_dag = Dagger(trial_func)
    trial = trial_func_dag * H_valued * trial_func
    pr = -1*abs(((qapply(trial).tolist())[0])[0])   # *(-1) をつけなければなりません
    # print(pr)
    f.write(str(count)+ ' ' +str(pr)+ ' ' +str(pr/E_answer)+ '\n')
    f.flush()
    return pr

In [ ]:
count = 0
f = sys.stdout
for i in range(10):
    vqe_trial(dice(6))

In [ ]:
count = 0
f = open('result_VQE.txt', 'w')
res = scipy.optimize.minimize(vqe_trial,dice(6),options={"maxiter": 100},method='Powell')
f.close()

In [ ]:
#pprint(res)
print(res["fun"])

In [ ]:
dat = np.loadtxt("result_VQE.txt")
plt.plot(dat[:,0],dat[:,1])
plt.plot(dat[:,0],dat[:,2])